# CCTV video processing

__<div style="text-align: right"> EE370: Software lab, Kyung Hee University. </div>__
_<div style="text-align: right"> Jong-Han Kim (jonghank@khu.ac.kr) </div>_

In this section, we will use OpenCV, one of the most popularly used image and video processing libraries, for extracting useful information form an RGB video clip obtained from a CCTV system.

Throught this exercise, you may take full advantages from using the OpenCV library (https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html) for whatever operation you may have to do.



We will access data files which are uploaded on the Google drive. In order to do this, we will have to mount the Goodle drive first. 

1) Run the following cell. Then you will be presented with a link and will be asked to enter your authorization code.

2) Click on the link to log in again with your KHU account, which you are working with. Then you will be presented with the authorization code.

3) Copy the authorization code, and paste it into the blank.

> _Note that this step is necessary ONLY when you work on Google Colab environment._



In [45]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/ee370')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The following will load a video clip `vtest.avi` from your local directory. 

In [46]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


cap = cv2.VideoCapture('vtest.avi')

For your information, the frame size and the frame rate can be obtained from the following.

In [47]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

print (width, height, fps)

768 576 10.0


Now your job is to detect moving persons wearing a red clothing. For example, for every frame from the clip like below,

<center>
<img src="https://jonghank.github.io/ee370/files/cctv1.jpg" width="600">
</center>

you have to highlight moving persons with a red clothing

<center>
<img src="https://jonghank.github.io/ee370/files/cctv2.jpg" width="600">
</center>


You may need to do the followings.

- Look around the whole pages (https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html).

- Separate the foreground (moving persons) from the background, for detecting moving objects (https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_bg_subtraction/py_bg_subtraction.html).

- Convert the frames to HSV scale, and carefully define the range of red colors in HSV scale, so that the defined bounds easily detect the red clothings.
to get masks for detecting red-ish pixels (https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_colorspaces/py_colorspaces.html). 

- Download and play the processed `output.avi` to check if everything was done as what you intended.

In [135]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow #imshow

cap = cv2.VideoCapture('vtest.avi') 
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG() #배경 제거

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'DIVX')
filename = 'output.avi'
out = cv2.VideoWriter(filename, fourcc, fps, (width, height)) #output.avi 설정

while(1):
    # Take each frame
    ret, frame = cap.read()

    if frame is None: #frame이 더 이상 없으면 실행되지 않도록.
      break;

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    fgmask = fgbg.apply(hsv) #hsv 상에서 배경 제거
      
    # define range of red color in HSV
    lower_red1 = np.array([0,150,0])
    upper_red1 = np.array([10,255,255])

    lower_red2 = np.array([170,150,0])
    upper_red2 = np.array([180,255,255])
      
    # Threshold the HSV image to get only red colors
    mask = fgmask * (cv2.inRange(hsv, lower_red1, upper_red1)+cv2.inRange(hsv, lower_red2, upper_red2))
      
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask) #이땐 제거가 안된다.

    out.write(res)

    k = cv2.waitKey(5) & 0xFF
    if k == 27:
      break

cap.release()
out.release()
cv2.destroyAllWindows()